<h1 style="background-color:#FF7733;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
📐 תהליך הסבת שכבת חזיתות בתהליך לבנק"ל מודרני 📐 

In [ ]:
print('13. Fronts In-Process:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import geopandas as gpd
import subprocess

In [ ]:
print('  13.a Reading inputs')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
RawData_folder = r"\\mapi_shares\MNCDB\צוות מידע\RawData"

<h1 style="text-align:right;font-size:100%">
שכבת חזיתות בתהליך קיימת

In [ ]:
fronts_inprocess_cols = ['FRONT_LEGAL_LENGTH', 'CUR_RADIUS', 'ENTITY_STATE', 'geometry']

fronts_inprocess = gpd.read_file(RawData_folder + '\CADSDE\Lines_InProcess.gdb',
                                 layer = 'Lines_InProcess',
                                 include_fields = fronts_inprocess_cols)

del fronts_inprocess_cols

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('  13.b Cleaning data')

In [ ]:
print('  13.c Data conversion')

In [ ]:
LegalLength_condition = (fronts_inprocess['FRONT_LEGAL_LENGTH'].isnull()
                                             |
                         fronts_inprocess['FRONT_LEGAL_LENGTH'] == 0)

fronts_inprocess['LegalLength'] = np.where(LegalLength_condition,
                                           np.nan,
                                           fronts_inprocess['FRONT_LEGAL_LENGTH'])


fronts_inprocess.drop(columns='FRONT_LEGAL_LENGTH', inplace=True)

In [ ]:
Radius_condition = (fronts_inprocess['CUR_RADIUS'].isnull()
                                    |
                    fronts_inprocess['CUR_RADIUS'] == 0)

fronts_inprocess['Radius']  = np.where(Radius_condition,
                                       0,
                                       fronts_inprocess['CUR_RADIUS'])

In [ ]:
fronts_inprocess['LineType'] = np.where(Radius_condition,
                                         1,
                                         2)


fronts_inprocess.drop(columns='CUR_RADIUS', inplace=True)
del [LegalLength_condition, Radius_condition]

In [ ]:
fronts_inprocess['LineStatus'] = np.nan
fronts_inprocess.loc[fronts_inprocess['ENTITY_STATE'] == 0, 'LineStatus'] = 1
fronts_inprocess.loc[fronts_inprocess['ENTITY_STATE'] == 1, 'LineStatus'] = 2

fronts_inprocess.drop(columns='ENTITY_STATE', inplace=True)

In [ ]:
fronts_inprocess['FrontUniqueID'] = range(1, 1 + len(fronts_inprocess))


fronts_inprocess['StartPointUniqueID'] = 0


fronts_inprocess['EndPointUniqueID'] = 0


fronts_inprocess['LeftParcelUniqueID'] = 0


fronts_inprocess['RightParcelUniqueID'] = 0

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
 print('  13.d Defining new Layer')

In [ ]:
fronts_inprocess = fronts_inprocess.astype({'FrontUniqueID'       : int,
                                            'LineType'            : int,
                                            'LineStatus'          : 'Int64',
                                            'Radius'              : float,
                                            'LegalLength'         : float,
                                            'StartPointUniqueID'  : int,
                                            'EndPointUniqueID'    : int,
                                            'LeftParcelUniqueID'  : int,
                                            'RightParcelUniqueID' : int,
                                            'geometry'            : 'geometry'})

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;"> 
<h1 style="text-align:right; font-size:100%;"> 
בחירת נתיב
<h1 style="text-align:right; font-size:100%;"> 
שמירה כטבלה
<h1 style="text-align:right; font-size:100%;"> 
שמירה כשכבה גיאוגרפית

In [ ]:
print('  13.e Exporting results to product folder')

In [ ]:
#csv_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\csv\FrontsInProcess.csv'
json_dir = r'\\mapi_shares\MNCDB\צוות מידע\תוצרים\json\InProcessFronts.geojson'


#fronts_inprocess.drop(columns='geometry').to_csv(csv_dir ,encoding='utf8', index=False)
fronts_inprocess.to_file(json_dir, driver = 'GeoJSON')

In [ ]:
subprocess.call([r"M:\ncdbScripts\scripts\arcpyPro\Fronts In-Process-Save as Feature Class.bat"]);